In [1]:
import pandas as pd
import numpy as np
import boto3
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import io

In [2]:
current_date = datetime.today().date()
LOOKBACK_DATE = (current_date - pd.DateOffset(months=18)).replace(day=1).strftime("'%Y-%m-%d'")

STEPS TO GENERATE CLAIM SUBMISSION FILE AFTER REVIEW COMPLETED

1. Load raw file from business team
2. Filter the loaded files only that are selected
3. Concatinate all filtered files into single dataframe
4. Check if selections are correct
5. Start Generation of claim submission file
6. Get the last key: Use claim submission table (use SQL query which groups claim submissions table for each 'vendor code' and 'program code'
    and max of last 6 letters of 'vendor audit id')
7. Generate claim submission for for selections


# 1. PROF1,OP1 Load raw file from business team
Load the claim submission files here, We may have multiple files:
1. Selection file generated from Lambda
2. Selection file generated from Manual selections

In [3]:
df_lambda  = pd.read_excel(r"D:\Selections July Cleanup\SCRIPTS TO RUN AFTER SELECTIONS IS DONE\Script to generate Claim Submission file\business_selections_profee_selection_file_for_review_07022025.xlsx", dtype=str)
df_manual  = pd.read_excel(r"D:\Selections July Cleanup\Run manual queries script\profee_manual_selection_file_for_review_07012025.xlsx", dtype=str)

# OP1 Load

In [4]:
# df_op_lambda = pd.read_excel(r"D:\Selections July Cleanup\SCRIPTS TO RUN AFTER SELECTIONS IS DONE\Script to generate Claim Submission file\business_selection_profee_ptp_mod_selection_file_for_review_07022025.xlsx", dtype=str)
df_op_manual = pd.read_excel(r"D:\Selections July Cleanup\SCRIPTS TO RUN AFTER SELECTIONS IS DONE\Script to generate Claim Submission file\business_selection_profee_ptp_mod_selection_file_for_review_07022025.xlsx", dtype=str)


In [6]:
# combine files for each program types in here

df_profee = pd.concat([df_lambda, df_manual])
df_op = df_op_manual.copy()



# 1. IP1 Load claims file after applying provider limitation

In [8]:
# df_ip= pd.read_excel()

df_ip = pd.read_csv(r"C:\Users\hamupratik\Downloads\selection_IP1_prov_excl_applied_2025_07_02.csv", dtype = str)

In [ ]:
df_ip['Rank'] = df_ip['Rank'].astype(int)
df_ip = df_ip.sort_values(by='Rank').reset_index(True)

In [9]:
only_required_columns_for_generation = ['query_name','template_name','adjudication_record_locator', 'member_record_locator', 
                                        'first_service_date', 'payer_control_number','plan_paid_amount','is_final_paid_indicator', 
                                        'bill_type_code', 'payment_effective_date','isselected_by_vendor', 
                                         'root_payer_control_number']

# IP claim submission file generation

In [10]:
print(f'IP unique pcn before dropping duplicated {df_ip.payer_control_number.nunique()}')
df_ip1 = df_ip[['adjudication_record_locator', 'member_record_locator', 'first_service_date', 'payer_control_number']].drop_duplicates().reset_index(drop= True)
print(f'IP unique pcn after dropping duplicated {df_ip1.payer_control_number.nunique()}')


IP unique pcn before dropping duplicated 814
IP unique pcn after dropping duplicated 814


Run this query in athena to get last key

In [11]:
"""
SELECT distinct "vendor code",
	"program code",
	max(
		cast(substring("vendor audit id", -6) as integer)
	)
FROM "devoted_health_prod"."claim_submissions"
group by "vendor code",
	"program code"   
"""

'\nSELECT distinct "vendor code",\n\t"program code",\n\tmax(\n\t\tcast(substring("vendor audit id", -6) as integer)\n\t)\nFROM "devoted_health_prod"."claim_submissions"\ngroup by "vendor code",\n\t"program code"   \n'

Assign variables for IP1

In [12]:
ip_btc='11' 
ip_vendor_code = 'ZIG' 
ip_vendor_concept_id = 'DRG1' 
ip_ref_claim_num = '' 
ip_opportunity_amount = '' 
ip_finding_reason_code = '' 
ip_audit_comment = '' 
ip_notes = ''
ip_key_pref = 'ZAUDEVIPDRG'
ip_key_suf = '' 
ip_last_key = 14196
ip_program_code = "IP1"

Assign variables for Prof1

In [13]:
profee_btc='' 
profee_vendor_code = 'ZIG' 
profee_vendor_concept_id = 'BUND1' 
profee_ref_claim_num = '' 
profee_opportunity_amount = '' 
profee_finding_reason_code = '' 
profee_audit_comment = '' 
profee_notes = ''
profee_key_pref = 'ZAUDEVPROF'
profee_key_suf = '' 
profee_last_key = 921
profee_program_code = "PROF1"


In [14]:
op_btc='13' 
op_vendor_code = 'ZIG' 
op_vendor_concept_id = '' #need to update 
op_ref_claim_num = '' 
op_opportunity_amount = '' 
op_finding_reason_code = '' 
op_audit_comment = '' 
op_notes = ''
op_key_pref = 'ZAUDEVOP'
op_key_suf = '' 
op_last_key = 850
op_program_code = "OP1"

In [15]:
def get_claim_submission_file_for_selections(df, vendor_code, program_code, vendor_concept_id, ref_claim_num, 
                opportunity_amount, finding_reason_code, audit_comment, notes, key_pref, key_suf, last_key):
    
    try:
        df1 = df[['adjudication_record_locator', 'member_record_locator', 'first_service_date', 
                                        'payer_control_number']].drop_duplicates()
        df1 = df1.reset_index(drop=True)
        df1['Vendor Code'] = vendor_code
        df1['Program Code'] = program_code
        df1['Vendor Concept ID'] = vendor_concept_id
        df1['Reference Claim Number'] = ref_claim_num
        df1['Opportunity Amount'] = opportunity_amount
        df1['Finding Reason Code'] = finding_reason_code
        df1['Audit Comments'] = audit_comment
        df1['NOTES'] = notes
    except Exception as e:
        print("GOT ERROR WHILE ADDING REQUIRED COLUMNS")
        print(e)

    try:
        df1['A'] = key_pref
        df1['B'] = key_suf       
        for i in range(len(df1['adjudication_record_locator'].unique())):
            df1.loc[[i], 'B'] = last_key+i+1
        df1['Vendor Audit ID'] = df1.apply(lambda x: x['A']+str(x['B']).zfill(6), axis=1)
        df1.drop(columns = ['A', 'B', 'payer_control_number'], inplace =True)
        rename1 = {'adjudication_record_locator':'Adjudication Record Locator', 'member_record_locator':'Member Record Locator',
           'first_service_date':'FDOS'}
        df1.rename(columns = rename1, inplace=True)
        df1 = df1[['Vendor Code', 'Adjudication Record Locator', 'Vendor Audit ID', 'Program Code', 
                                     'Vendor Concept ID', 'Member Record Locator', 'FDOS', 'Reference Claim Number', 
                                     'Opportunity Amount','Finding Reason Code', 'Audit Comments', 'NOTES']]
        return df1
    except Exception as e:
        print("GOT ERROR WHILE GENERATING VENDOR AUDIT ID")
        print(e)

Run claims submission function with IP variables

In [16]:
pipe_ip = get_claim_submission_file_for_selections(df_ip1, ip_vendor_code, ip_program_code, ip_vendor_concept_id, ip_ref_claim_num, 
                ip_opportunity_amount, ip_finding_reason_code, ip_audit_comment, ip_notes, ip_key_pref, ip_key_suf, ip_last_key)
 


Run claims submission function with profee variables

In [20]:
pipe_profee = get_claim_submission_file_for_selections(df_profee, profee_vendor_code, profee_program_code, profee_vendor_concept_id, profee_ref_claim_num, 
                profee_opportunity_amount, profee_finding_reason_code, profee_audit_comment, profee_notes, profee_key_pref, profee_key_suf, profee_last_key)

Run claims submission function with OP variables

In [21]:
pipe_op = get_claim_submission_file_for_selections(df_op, op_vendor_code, op_program_code, op_vendor_concept_id, op_ref_claim_num, 
                op_opportunity_amount, op_finding_reason_code, op_audit_comment, op_notes, op_key_pref, op_key_suf, op_last_key)

In [22]:
pipe_ip.isna().sum()

Vendor Code                    0
Adjudication Record Locator    0
Vendor Audit ID                0
Program Code                   0
Vendor Concept ID              0
Member Record Locator          0
FDOS                           0
Reference Claim Number         0
Opportunity Amount             0
Finding Reason Code            0
Audit Comments                 0
NOTES                          0
dtype: int64

In [23]:
pipe_profee.isna().sum()

Vendor Code                    0
Adjudication Record Locator    0
Vendor Audit ID                0
Program Code                   0
Vendor Concept ID              0
Member Record Locator          0
FDOS                           0
Reference Claim Number         0
Opportunity Amount             0
Finding Reason Code            0
Audit Comments                 0
NOTES                          0
dtype: int64

In [24]:
pipe_op.isna().sum()

Vendor Code                    0
Adjudication Record Locator    0
Vendor Audit ID                0
Program Code                   0
Vendor Concept ID              0
Member Record Locator          0
FDOS                           0
Reference Claim Number         0
Opportunity Amount             0
Finding Reason Code            0
Audit Comments                 0
NOTES                          0
dtype: int64

Concat all program codes into one file

In [25]:
print(f'total claims IP: {pipe_ip.shape[0]}')

total claims IP: 814


In [26]:
print(f'total claims PROFEE: {pipe_profee.shape[0]}')

total claims PROFEE: 15


In [27]:
print(f'total claims OP: {pipe_op.shape[0]}')

total claims OP: 10


In [28]:
final_claim_sub_all_programs = pd.concat([pipe_ip,pipe_profee,pipe_op])
final_claim_sub_all_programs

,Vendor Code,Adjudication Record Locator,Vendor Audit ID,Program Code,Vendor Concept ID,Member Record Locator,FDOS,Reference Claim Number,Opportunity Amount,Finding Reason Code,Audit Comments,NOTES
0,ZIG,AJX6942UKZ,ZAUDEVIPDRG014197,IP1,DRG1,DG25Z9,2025-05-21,,,,,
1,ZIG,AJX6ZSSFR4,ZAUDEVIPDRG014198,IP1,DRG1,D2W65K,2025-06-11,,,,,
2,ZIG,AJXYF74GJH,ZAUDEVIPDRG014199,IP1,DRG1,DAE6K2,2025-06-11,,,,,
3,ZIG,AJXS8C57WJ,ZAUDEVIPDRG014200,IP1,DRG1,D57FC2,2025-03-27,,,,,
4,ZIG,AJXS274GZ5,ZAUDEVIPDRG014201,IP1,DRG1,DRJEUE,2025-05-13,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
5,ZIG,AJXR3JJ9HJ,ZAUDEVOP000856,OP1,,DEY7KK,2022-10-20,,,,,
6,ZIG,AJXWG9SUCG,ZAUDEVOP000857,OP1,,DS99W5,2024-02-08,,,,,
7,ZIG,AJXJAA3CW6,ZAUDEVOP000858,OP1,,D822RR,2023-06-14,,,,,
8,ZIG,AJXZS9G7K7,ZAUDEVOP000859,OP1,,DWEH8W,2024-01-17,,,,,


In [29]:
print(f'total claims: {final_claim_sub_all_programs.shape[0]}')

total claims: 839


In [30]:
for i in final_claim_sub_all_programs.columns:
    if (final_claim_sub_all_programs[i].isnull() | (final_claim_sub_all_programs[i].str.strip() == '')).any():
        print("null_columns:", i)

null_columns: Vendor Concept ID
null_columns: Reference Claim Number
null_columns: Opportunity Amount
null_columns: Finding Reason Code
null_columns: Audit Comments
null_columns: NOTES


In [31]:
formatted_date = current_date.strftime("%Y%m%d")
formatted_date

'20250818'

In [32]:
file_name=f"ZIGIP1OP1PROF1toDevoted_Submission_{formatted_date}.txt"
out_put_dir=r"D:\temp"

In [33]:
fr"{out_put_dir}\{file_name}"

'D:\\temp\\ZIGIP1OP1PROF1toDevoted_Submission_20250818.txt'

# Final claim submission file

In [ ]:
final_claim_sub_all_programs.to_csv(fr"{out_put_dir}\{file_name}", sep = '|', index= False)

END

# Selections table

In [34]:
df_profee['template_name'] = 'PROFEE'
df_profee[['adjudication_record_locator','query_name', 'template_name']]

# df_profee.to_excel()

,adjudication_record_locator,query_name,template_name
0,AJXGZHASA8,prof_cci_medadv_anterior_instrumentation,PROFEE
1,AJXGZHASA8,prof_cci_medadv_anterior_instrumentation,PROFEE
2,AJXGZHASA8,prof_cci_medadv_anterior_instrumentation,PROFEE
3,AJXGZHASA8,prof_cci_medadv_anterior_instrumentation,PROFEE
4,AJX2487H7S,prof_cci_medadv_anterior_instrumentation,PROFEE
...,...,...,...
61,AJXZ459A87,prof_cci_medadv_anterior_instrumentation,PROFEE
62,AJXZ459A87,prof_cci_medadv_anterior_instrumentation,PROFEE
0,AJXGCA5W28,all_bundle_medadv_32408,PROFEE
1,AJXGCA5W28,all_bundle_medadv_32408,PROFEE


In [ ]:
df_op['template_name'] = 'OP1'
df_op[['adjudication_record_locator', 'query_name', 'template_name']]

# df_op.to_excel()

,adjudication_record_locator,query_name,template_name
0,AJXG6GS2GK,prof_bundle_medadv_ptp,OP1
1,AJXG6GS2GK,prof_bundle_medadv_ptp,OP1
2,AJXG6GS2GK,prof_bundle_medadv_ptp,OP1
3,AJXG6GS2GK,prof_bundle_medadv_ptp,OP1
4,AJXG6GS2GK,prof_bundle_medadv_ptp,OP1
...,...,...,...
109,AJXZUZRZUH,prof_bundle_medadv_ptp,OP1
110,AJXZUZRZUH,prof_bundle_medadv_ptp,OP1
111,AJXZUZRZUH,prof_bundle_medadv_ptp,OP1
112,AJXZUZRZUH,prof_bundle_medadv_ptp,OP1


IP selection table

In [ ]:
df_ip[['adjudication_record_locator','query_name', 'template_name']]
# df_ip.to_excel()

,adjudication_record_locator,query_name,template_name
0,AJX6942UKZ,drg_cva_medadv_1mcc_acutechf,MISC
1,AJX6942UKZ,drg_cva_medadv_1mcc_acutechf,MISC
2,AJX6ZSSFR4,drg_cva_medadv_1cc_abla,MISC
3,AJXYF74GJH,drg_cva_medadv_1mcc_ami,MISC
4,AJXYF74GJH,drg_cva_medadv_1mcc_ami,MISC
...,...,...,...
2085,AJXK5CZ7CF,drg_cva_medadv_1mcc_generalrespfailure,MISC
2086,AJXK5CZ7CF,drg_cva_medadv_1mcc_generalrespfailure,MISC
2087,AJXZ9KRH22,"drg_coding_medadv_871_sepsis_diverticultis, dr...",SEPSIS
2088,AJXZ9KRH22,"drg_coding_medadv_871_sepsis_diverticultis, dr...",SEPSIS
